## Sistema: n servidores en serie con retroceso

Este proyecto se enfoca en la creación de una simulación de eventos discretos con el fin de analizar y comprender diversos fenómenos. Nuestro objetivo es aplicar los principios de esta simulación para modelar y experimentar con dichos fenómenos, con el propósito de obtener resultados que guíen nuestras decisiones de manera informada.

Los clientes llegan a un sistema que tiene n servidores, y las llegadas distribuye M. Cada cliente que llega debe ser atendido primero por el servidor 1 y, al completar el servicio en el servidor 1, el cliente pasa al servidor 2.

Cuando un cliente llega, entra en servicio con el servidor 1 si ese servidor está libre, o se une a la cola del servidor 1 en caso contrario. De manera similar, cuando el cliente completa el servicio en el servidor 1, entra en servicio con el servidor 2 si ese servidor está libre, o se une a su cola y asi sucesivamente. Después de ser atendido en el servidor n, el cliente abandona el sistema.

El servidor i con una probabilidad p pude se salta a la cola del servidor j.

Los tiempos de servicio en el servidor i tienen la distribución Gi

### Variables

#### Tiempo

$t$: tiempo general  
$t_A$: tiempo de arribo del proximo cliente  
$t_Di$: tiempo de salido del servidor i  

#### Contadoras

$n_A$: cantidad de arribos  
$n_D$: cantidad de salidas  
$A_i$: arribos al servidor i  
$D_j$: salidas del servidor i  

#### Estado

$Queue_i$: cola de espera del servidor i


In [ ]:
from enum import Enum
import math
from queue import PriorityQueue as pq
import random

class EventType(Enum):
    ASIGNED = 1
    ARRIVAL = 2
    FINISH = 3

def generate_exponential(lambd): 
    return math.log(1 - random.uniform(), math.e) / -lambd

def generate_bernoulli(p):
    return 1 if random.uniform() < p else 0

def generate_poisson(lambd):
    L = math.exp(-lambd)
    k = 0
    p = 1

    while True:
        k += 1
        p *= random.uniform(0, 1)
        if p <= L:
            return k - 1

def simulation(n, lambd, p):
    t = 0
    events = pq()
    n_a = 0
    n_d = 0
    a = [{} for _ in range(n)]
    d = [{} for _ in range(n)]
    queue = [[] for _ in range(n)]

    while(True):
        events.put((generate_exponential(lambd), EventType.ASIGNED, 0, 0))
        t, e, c, i = events.get()
        if e == EventType.ASIGNED:
            if len(queue[i]) == 0:
                events.put((t, EventType.ARRIVAL, c if i !=0 else n_a, i))
            else:
                queue[i].append[c if i !=0 else n_a] 
            if i == 0:
                n_a += 1
        elif e == EventType.ARRIVAL:
            a[i][c] = t
            duration = generate_exponential(lambd)
            events.put((t + duration, EventType.FINISH, c, i))
            if queue[i] and len(queue[i]) > 1:
                events.put((t + duration, EventType.ARRIVAL, queue[i][1], i))
        elif e == EventType.FINISH:
            queue[i].pop(0)
            d[i][c] = t
            if i == n-1:
                n_d += 1
                continue

            events.put((t, EventType.ASIGNED, c, i+random.randint(1, i-1 if generate_bernoulli(p) == 1 else i+1)))


